In [ ]:
project_directory = r"C:\Users\abjawad\Documents\GitHub\carla-roach-0.9.13"
import sys
sys.path.append(project_directory)

In [ ]:
import xml.etree.ElementTree as ET
import carla
import yaml
import os
from utils.server_utils import CarlaServerManager
from carla_gym.core.task_actor.common.navigation.global_route_planner import GlobalRoutePlanner


In [ ]:
project_directory = r"C:\Users\abjawad\Documents\GitHub\carla-roach-0.9.13"

In [ ]:
def create_route_dict(file_path):
    # Parse XML file    
    tree = ET.parse(file_path)
    root = tree.getroot()
    # Dictionary to store route data
    routes_dict = {}
    for route in root:  # Each 'route' element
        route_id = route.attrib['id']
        waypoints = []
        for vehicle in route:  # Each 'ego_vehicle' within the route
            for waypoint in vehicle:  # Each 'waypoint' within the vehicle
                waypoint_coords = carla.Location(
                    x=float(waypoint.attrib['x']),
                    y=float(waypoint.attrib['y']),
                    z=float(waypoint.attrib['z'])
                )
                waypoints.append(waypoint_coords)  
        if waypoints:  # If there are waypoints found
            routes_dict[route_id] = waypoints  # Store start and destination
    return routes_dict

# def route_distance(file_path):
#     route_dict = create_route_dict(file_path)
#     accumulated_distance = 0
#     for route_id, waypoints in route_dict.items():
#         total_distance = 0
#         for i in range(len(waypoints)-1):
#             distance = carla.Location(waypoints[i]).distance(carla.Location(waypoints[i+1]))
#             total_distance += distance
#         print(f"Route {route_id}: {total_distance} meters")
#         accumulated_distance += total_distance
#     print(f"Total distance: {accumulated_distance} meters")
#     pass


def read_config(config_path):
    with open(config_path, 'r') as file:
        cfg = yaml.safe_load(file)
    return cfg

def init_client(host, port):
    try:
        client = carla.Client(host, port)
        print(f"client connected to {host}:{port}")
        print(f"client.get_server_version(): {client.get_server_version()}, client.get_client_version(): {client.get_client_version()}")
        client.set_timeout(60.0)
    except RuntimeError as re:
        if "timeout" not in str(re) and "time-out" not in str(re):
            print("Could not connect to Carla server because:", re)
        client = None 
    return client



def draw_route(route, frequency=15):
    for i, waypoint in enumerate(route):
        if i % frequency == 0:
            debug.draw_string(location=waypoint[0].transform.location,
                            text=str(waypoint[0].transform.location),
                            color=carla.Color(0,0,0),
                            life_time=1000,
                            persistent_lines=True)
            debug.draw_point(location=waypoint[0].transform.location,
                            size=0.1,
                            color=carla.Color(0,0,255),
                            life_time=1000)


In [ ]:
routes = r"carla_gym\envs\scenario_descriptions\LeaderBoard\Town05\routes.xml"
routes = os.path.join(project_directory, routes)

config_path = r'config\replay.yaml'
config_path = os.path.join(project_directory, config_path)

In [ ]:
routes_dict = create_route_dict(routes)

In [ ]:
cfg = read_config(config_path)
server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
# server_manager.stop()
# server_manager.start()

In [ ]:
client = init_client(cfg['host'], cfg['port'])

# load map Town04
world = client.load_world('Town04')
world = client.get_world()

In [ ]:
# from utils import analysis_utils as autils
# import pandas as pd

# data_folders = {

#     'seed1' : os.path.join(project_directory, 'outputs', 'experiment_1', '23-40-09'), # '2024-04-26'
#     'seed2' : os.path.join(project_directory, 'outputs', 'experiment_1', '03-03-02'), # '2024-04-26'
#     'seed3' : os.path.join(project_directory, 'outputs', 'experiment_1', '00-14-25'), # '2024-04-27'
#     'seed4' : os.path.join(project_directory, 'outputs', 'experiment_1', '00-06-31'), # '2024-04-28'
# }
# # file_path = data_folders['seed1']
# # df = autils.create_episode_df(file_path)
# # print(len(df))
# # autils.plot_accidents_vs_traffic_density(df)

# file_paths = list(data_folders.values())
# collisions = pd.DataFrame()
# for file_path in file_paths:
#     df = autils.create_df_for_collisions_with_moving_objects(file_path)
#     collisions = pd.concat([collisions, df])

# print(len(collisions))
# collisions.columns

In [ ]:
# town04_collisions = collisions[collisions['location'] == 'Town04']
# print(len(town04_collisions))
# # create list of carla locations using the col 'event_loc_x', 'event_loc_y','event_loc_z'
# locations = []
# for i, row in town04_collisions.iterrows():
#     location = carla.Location(x=row['event_loc_x'], y=row['event_loc_y'], z=row['event_loc_z'])
#     locations.append(location)
# print(len(locations))

# debug = world.debug
# for loc in locations:
#     debug.draw_point(loc, size=0.1, color=carla.Color(255,0,0), life_time=1000)
# spectator = world.get_spectator()

# # make deep copy of the first point
# spec_loc = carla.Location(locations[0])
# spec_loc.z += 40

# # set spectator to the first point
# spectator.set_transform(carla.Transform(spec_loc, carla.Rotation(pitch=-90)))


In [ ]:
# print(routes_dict.keys())

route = routes_dict['16']
# print(route)

# # draw a point in carla
debug = world.debug
# for i in range(len(route)-1):
#     print(route[i])
#     debug.draw_line(begin=route[i],
#                      end=route[i+1],
#                     thickness=1,
#                     color=carla.Color(255,0,0),
#                     life_time=1000)

# set spectator to the first point with looking downward from 40 m above
spectator = world.get_spectator()

# make deep copy of the first point
spec_loc = carla.Location(route[-2])
spec_loc.z += 40

# set spectator to the first point
spectator.set_transform(carla.Transform(spec_loc, carla.Rotation(pitch=-90)))



In [ ]:
# draw text in carla for each waypoint in the route
for i, waypoint in enumerate(route):
    debug.draw_string(location=waypoint,
                      text=str(i),
                      color=carla.Color(255,0,0),
                      life_time=1000,
                      persistent_lines=True)

In [ ]:
planner = GlobalRoutePlanner(world.get_map(), 1)
# start = carla.Location(route[0])
# destination = carla.Location(route[-1])
# trace_route = planner.trace_route(start, destination)

# take consecutive point from route and use trace_route to get the route
for i in range(len(route)-1):
    start = carla.Location(route[i])
    destination = carla.Location(route[i+1])
    trace_route = planner.trace_route(start, destination)
    draw_route(trace_route, frequency=4)